<a href="https://colab.research.google.com/github/Tar-ive/ML-projects/blob/main/Q_A_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 140.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    Trainer,
    TrainingArguments,
    default_data_collator
)

In [3]:
df = pd.read_csv("questions_data.csv")


In [4]:
df

,context,question,answers.text,answers.answer_start
0,You are a 19 year old international student fr...,Why do you want to study in the USA?,I want to study in the USA because it offers e...,0
1,You are a 19 year old international student fr...,What is your major? Why do you want to study t...,My major is finance. I have a strong interest ...,0
2,You are a 19 year old international student fr...,Why do you choose this particular university?,I chose this particular university because of ...,0
3,You are a 19 year old international student fr...,How many universities did you apply to?,"I applied to five universities in total, caref...",0
4,You are a 19 year old international student fr...,What are your plans after graduation?,"After graduation, I plan to return to Nepal an...",0
5,You are a 19 year old international student fr...,"Do you have any academic gap year? If yes, wha...","No, I do not have any academic gap years. I ha...",0
6,You are a 19 year old international student fr...,Do you have any friends or relatives in the USA?,I do not have any friends or relatives current...,0
7,You are a 19 year old international student fr...,"Do you have any siblings? If yes, what do they...","Yes, I have a brother who runs his own startup...",0
8,You are a 19 year old international student fr...,Who is going to be your sponsor?,My sponsor will be my father. He is the editor...,0
9,You are a 19 year old international student fr...,What does your sponsor do for a living and wha...,"My father, who is my sponsor, is the editor of...",0


In [7]:
!pip install torch 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import torch
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForQuestionAnswering

In [9]:
questions = df['question'].tolist()
contexts = df['context'].tolist()
answers = df['answers.text'].tolist()
answer_starts = df['answers.answer_start'].tolist()

examples = []
for question, context, answer, answer_start in zip(questions, contexts, answers, answer_starts):
    examples.append({
        'question': question,
        'context': context,
        'answer_text': answer,
        'answer_start': answer_start
    })

In [10]:
from transformers import BertForQuestionAnswering, BertTokenizer
import pandas as pd


In [12]:
model_name = 'bert-base-uncased'  # Choose a BERT model from Hugging Face's library
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [13]:
def preprocess_data(row):
    context = row['context']
    question = row['question']
    answer = row['answers.text']
    start_pos = row['answers.answer_start']
    
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors='pt')
    input_ids = inputs['input_ids'].flatten()
    token_type_ids = inputs['token_type_ids'].flatten()
    
    start_positions = torch.tensor([start_pos])
    end_positions = torch.tensor([start_pos + len(answer)])
    
    return {
        'input_ids': input_ids,
        'attention_mask': (input_ids != 0).long(),  # Mask padding tokens
        'token_type_ids': token_type_ids,
        'start_positions': start_positions,
        'end_positions': end_positions
    }

# Apply the preprocessing function to each row of the DataFrame
dataset = df.apply(preprocess_data, axis=1)


In [15]:
from torch.utils.data import DataLoader, Dataset

from torch.nn.utils.rnn import pad_sequence

class QADataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_masks = [item['attention_mask'] for item in batch]
    token_type_ids = [item['token_type_ids'] for item in batch]
    start_positions = torch.tensor([item['start_positions'] for item in batch])
    end_positions = torch.tensor([item['end_positions'] for item in batch])

    input_ids = pad_sequence(input_ids, batch_first=True)
    attention_masks = pad_sequence(attention_masks, batch_first=True)
    token_type_ids = pad_sequence(token_type_ids, batch_first=True)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'token_type_ids': token_type_ids,
        'start_positions': start_positions,
        'end_positions': end_positions
    }

# Create a DataLoader for training
train_dataset = QADataset(dataset)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)


# Set up training parameters
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
model.train()
for epoch in range(3):  # Choose the number of training epochs
    total_loss = 0
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    print(f"Epoch {epoch+1} - Average Loss: {total_loss/len(train_dataloader):.4f}")


Epoch 1 - Average Loss: nan
Epoch 2 - Average Loss: nan
Epoch 3 - Average Loss: nan


In [16]:
model.save_pretrained('path_to_save_model')
tokenizer.save_pretrained('path_to_save_tokenizer')


('path_to_save_tokenizer/tokenizer_config.json',
 'path_to_save_tokenizer/special_tokens_map.json',
 'path_to_save_tokenizer/vocab.txt',
 'path_to_save_tokenizer/added_tokens.json')

In [19]:
model_name = 'bert-base-uncased'
model = BertForQuestionAnswering.from_pretrained('model_name')
tokenizer = BertTokenizer.from_pretrained('model_name')

# Provide a new question and context
question = "What is the capital of France?"
context = "France, officially the French Republic, is a country whose capital is Paris."

# Preprocess the new input
inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors='pt')
input_ids = inputs['input_ids'].flatten()
token_type_ids = inputs['token_type_ids'].flatten()

# Pass the input through the model
inputs = {
    'input_ids': input_ids.to(device),
    'attention_mask': (input_ids != 0).long().to(device),
    'token_type_ids': token_type_ids.to(device)
}
outputs = model(**inputs)

# Get the predicted answer
start_pos = torch.argmax(outputs.start_logits)
end_pos = torch.argmax(outputs.end_logits)
answer = tokenizer.decode(inputs['input_ids'][0, start_pos:end_pos+1])

print(f"Question: {question}")
print(f"Answer: {answer}")


OSError: ignored